In [1]:
#Importing the data labelled as excel file
import pandas as pd
data=pd.read_excel("DLI_CO_DATA.xlsx")
data=data.iloc[:,[0,1,2,3,4,5,6]]
#Checking the first five rows of data
data.tail()

,Date/Time,NO2 (μg/m3),SO2 (μg/m3),O3 (μg/m3),CO (μg/m3),PM10 (μg/m3),PM 2.5 (μg/m3)
7531,2020-11-09 19:00:00,123.66430,6.942140,2.626272,2626.576,112.54680,34.47053
7532,2020-11-09 20:00:00,102.62100,7.881799,2.665070,2744.537,185.77910,51.01247
7533,2020-11-09 21:00:00,83.50191,6.309392,1.787648,2180.894,152.05290,50.29734
7534,2020-11-09 22:00:00,65.23309,5.243093,1.287961,1759.910,122.83820,48.03581
7535,2020-11-09 23:00:00,56.35237,3.984219,1.077103,1304.964,94.77707,41.15864


In [2]:
data.interpolate()
data.shape

(7536, 7)

In [3]:
# Deriving the two columns: Date/Time and CO
data=data.iloc[:,[0,4]]
data.columns=['Date','CO']
#Printing the date upto period 'Day'
data['Date'] = pd.to_datetime(data['Date']).dt.to_period('H')
#Drops all rows with missing values
#data=data.dropna()
#Setting the date column as an index column to allow resampling
data=data.set_index('Date')
#Resampling data to give only the mean daily CO concentrations
data=data.resample('D').mean()
data['date'] = data.index
#data['date'] = pd.to_datetime(data['date']).dt.to_period('D')
mean_value=data['CO'].mean()
data['CO'].fillna(value=data['CO'].mean(), inplace=True)
data.isnull().values.any()
data.head()

,CO,date
Date,,
2020-01-01,894.455633,2020-01-01
2020-01-02,577.588292,2020-01-02
2020-01-03,402.879125,2020-01-03
2020-01-04,539.677329,2020-01-04
2020-01-05,704.075854,2020-01-05


In [4]:
#Deriving the lists of Date and CO columns of the data
Date=data['date'].to_list()
CO  =data['CO'].to_list()
date=[i+1 for i in range(len(CO)) ]
#Generating timeseries data based on the two lists
data= pd.Series(CO,Date)
CO=pd.Series(CO)
data

2020-01-01    894.455633
2020-01-02    577.588292
2020-01-03    402.879125
2020-01-04    539.677329
2020-01-05    704.075854
                 ...    
2020-11-05    426.697750
2020-11-06    782.375809
2020-11-07    574.743696
2020-11-08    494.527079
2020-11-09    908.988788
Freq: D, Length: 314, dtype: float64

In [5]:
from sktime.datasets import load_airline
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.performance_metrics.forecasting import smape_loss
from sktime.utils.plotting import plot_series
from warnings import simplefilter

import numpy as np

simplefilter("ignore", FutureWarning)

y = data
# our HCrystalball wrapper currently only supports pd.DatetimeIndex types
y.index = y.index.to_timestamp()
y_train, y_test = temporal_train_test_split(y, test_size=71)
print(y_train.shape[0], y_test.shape[0])

ImportError: cannot import name 'plot_series' from 'sktime.utils.plotting' (/Users/kipkemoivincent/opt/anaconda3/lib/python3.8/site-packages/sktime/utils/plotting/__init__.py)

In [ ]:
from hcrystalball.ensemble import SimpleEnsemble
from hcrystalball.wrappers import (
    ExponentialSmoothingWrapper,
    SarimaxWrapper,
    get_sklearn_wrapper,
)
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet

from sktime.forecasting.hcrystalball import HCrystalBallForecaster

In [ ]:
models = {
    #"sarimax": SarimaxWrapper(init_with_autoarima=True, autoarima_dict={"m": 12}),
    "smoothing": ExponentialSmoothingWrapper(
        trend="add", seasonal="multiplicative", seasonal_periods=12
    ),
    "sklearn_linear": get_sklearn_wrapper(ElasticNet),
    "sklearn_forest": get_sklearn_wrapper(RandomForestRegressor, n_estimators=100),
    "avg_ensemble": SimpleEnsemble(
        base_learners=[
            get_sklearn_wrapper(ElasticNet, name="sklearn_linear"),
            ExponentialSmoothingWrapper(
                trend="add",
                seasonal="multiplicative",
                seasonal_periods=12,
                name="smoothing",
            ),
        ],
        ensemble_func="mean",
    ),
}

In [7]:
fh = np.arange(len(y_test)) + 1
for model_name, model in models.items():
    forecaster = HCrystalBallForecaster(model)
    # if provided in the data, exogenous variables are supported over X_train, X_pred
    forecaster.fit(y_train)
    y_pred = forecaster.predict(fh)
    plot_series(
        y_train,
        y_test,
        y_pred,
        labels=[
            "y_train",
            "y_test",
            f"y_pred_{model_name}[smape_loss:{smape_loss(y_pred, y_test):.3}]",
        ],
    )

NameError: name 'models' is not defined